# Debug Drill: The Diverging Neural Network

**Scenario:**
A colleague is training a neural network for customer churn prediction.

"The loss keeps going up instead of down!" they say, frustrated.

The model is diverging during training, producing NaN or exploding loss values.

**Your Task:**
1. Identify why the neural network is diverging
2. Fix the training issues
3. Get the model to converge properly
4. Write a 3-bullet postmortem

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

In [ ]:
# Generate customer churn data
X, y = make_classification(
    n_samples=1000, 
    n_features=20, 
    n_informative=10,
    n_redundant=5,
    random_state=42
)

# Make features on different scales (realistic)
X[:, 0] *= 1000    # Revenue: ~1000s
X[:, 1] *= 100     # Orders: ~100s
X[:, 2] *= 0.01    # Rate: ~0.01

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dataset: {len(X_train)} train, {len(X_test)} test")
print(f"Feature scales:")
print(f"  Feature 0 (Revenue): {X[:, 0].min():.0f} to {X[:, 0].max():.0f}")
print(f"  Feature 1 (Orders):  {X[:, 1].min():.0f} to {X[:, 1].max():.0f}")
print(f"  Feature 2 (Rate):    {X[:, 2].min():.4f} to {X[:, 2].max():.4f}")

In [ ]:
# ===== COLLEAGUE'S CODE (BUGS: No scaling, high learning rate) =====

# BUG 1: No feature scaling!
# BUG 2: Learning rate too high!

model_broken = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    learning_rate_init=1.0,      # <-- BUG: WAY too high!
    max_iter=100,
    random_state=42,
    verbose=False
)

# Try to train (will fail or produce bad results)
try:
    model_broken.fit(X_train, y_train)  # <-- NOT SCALED!
    train_acc = model_broken.score(X_train, y_train)
    test_acc = model_broken.score(X_test, y_test)
    final_loss = model_broken.loss_
    
    print("=== Colleague's Results ===")
    print(f"Final loss: {final_loss:.4f}")
    print(f"Train accuracy: {train_acc:.1%}")
    print(f"Test accuracy: {test_acc:.1%}")
    
    if final_loss > 1.0 or test_acc < 0.6:
        print("\n❌ Model appears to have diverged or failed to learn!")
except Exception as e:
    print(f"Training failed: {e}")

In [ ]:
# Visualize the loss curve (if training didn't crash)
if hasattr(model_broken, 'loss_curve_') and len(model_broken.loss_curve_) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(model_broken.loss_curve_, 'r-', linewidth=2)
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Loss Curve (Diverging or Unstable)')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print("🔍 Notice: Loss may be erratic, increasing, or stuck at a high value")
else:
    print("No loss curve available (training may have failed early)")

---

## Your Investigation

### Step 1: Identify the issues

In [ ]:
print("=== Bug Analysis ===")
print()
print("❌ BUG 1: No feature scaling")
print("   Neural networks are sensitive to feature scales")
print("   Features range from 0.01 to 1000 - gradients will be unstable")
print()
print("❌ BUG 2: Learning rate = 1.0 (WAY too high)")
print("   Typical learning rates: 0.001 to 0.01")
print("   High LR causes overshooting → divergence")
print()
print("Combined effect: Exploding gradients → NaN or stuck loss")

### Step 2: TODO - Fix the neural network

In [ ]:
# TODO: Fix the training issues

# Uncomment and complete:

# # Fix 1: Scale the features
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# 
# # Fix 2: Use a reasonable learning rate
# model_fixed = MLPClassifier(
#     hidden_layer_sizes=(64, 32),
#     learning_rate_init=0.001,   # Fixed: reasonable LR
#     max_iter=200,
#     random_state=42,
#     early_stopping=True,        # Added: prevent overfitting
#     validation_fraction=0.1
# )
# 
# model_fixed.fit(X_train_scaled, y_train)  # Train on SCALED data
# 
# print("=== Fixed Model Results ===")
# print(f"Final loss: {model_fixed.loss_:.4f}")
# print(f"Train accuracy: {model_fixed.score(X_train_scaled, y_train):.1%}")
# print(f"Test accuracy: {model_fixed.score(X_test_scaled, y_test):.1%}")

In [ ]:
# TODO: Compare loss curves

# Uncomment:

# fig, axes = plt.subplots(1, 2, figsize=(14, 5))
# 
# # Broken model
# if hasattr(model_broken, 'loss_curve_') and len(model_broken.loss_curve_) > 0:
#     axes[0].plot(model_broken.loss_curve_, 'r-', linewidth=2)
# axes[0].set_xlabel('Iteration')
# axes[0].set_ylabel('Loss')
# axes[0].set_title('BROKEN: Unscaled + High LR')
# axes[0].grid(True, alpha=0.3)
# 
# # Fixed model
# axes[1].plot(model_fixed.loss_curve_, 'g-', linewidth=2)
# axes[1].set_xlabel('Iteration')
# axes[1].set_ylabel('Loss')
# axes[1].set_title('FIXED: Scaled + Reasonable LR')
# axes[1].grid(True, alpha=0.3)
# 
# plt.tight_layout()
# plt.show()
# 
# print("\n✓ Fixed model shows smooth, decreasing loss!")

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

# Uncomment:

# test_acc_fixed = model_fixed.score(X_test_scaled, y_test)
# assert test_acc_fixed > 0.7, f"Fixed model should achieve >70% accuracy, got {test_acc_fixed:.1%}"
# assert model_fixed.loss_ < 0.5, f"Loss should be low, got {model_fixed.loss_:.3f}"
# 
# print("✓ Neural network fixed!")
# print(f"✓ Test accuracy: {test_acc_fixed:.1%}")
# print(f"✓ Final loss: {model_fixed.loss_:.4f}")

### Step 3: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Diverging Neural Network

### What happened:
- (Your answer: What symptoms indicated the model was diverging?)

### Root cause:
- (Your answer: What two issues caused the divergence?)

### How to prevent:
- (Your answer: What preprocessing and hyperparameters are essential for NNs?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **Always scale features for neural networks.** They're sensitive to input magnitudes.

2. **Start with small learning rates.** 0.001 is a safe default.

3. **Watch the loss curve.** Increasing or erratic loss = divergence.

4. **Use early stopping.** Prevents overfitting and saves time.

---

## Neural Network Checklist

| Check | Why |
|-------|-----|
| Scale features (StandardScaler) | NNs are scale-sensitive |
| Small learning rate (0.001) | Prevents divergence |
| Early stopping | Prevents overfitting |
| Monitor loss curve | Detect problems early |
| Check train/test gap | Detect overfitting |